# NOTICIAS

In [ ]:
import requests
import pandas as pd

# === CONFIGURACIÓN ===
API_KEY = ''
tema = 'economia'
fecha_desde = '2025-03-15'
fecha_hasta = '2025-03-20'
idioma = 'es'

# === URL API ===
url = f'https://newsapi.org/v2/everything?q={tema}&language={idioma}&from={fecha_desde}&to={fecha_hasta}&pageSize=100&apiKey={API_KEY}'

# === CONSULTA ===
response = requests.get(url)
data = response.json()

if data.get('status') == 'ok':
    articulos = data['articles']
    df = pd.DataFrame([{
        'fecha': art['publishedAt'],
        'titulo': art['title'],
        'autor': art['author']
    } for art in articulos])

    # === LIMPIEZA Y FORMATEO ===
    df['fecha'] = pd.to_datetime(df['fecha']).dt.date

    # === INDICADORES DE CALIDAD ===
    total = len(df)
    nulos_autor = df['autor'].isna().sum()
    nulos_titulo = df['titulo'].isna().sum()
    duplicados_titulo = df.duplicated(subset='titulo').sum()
    fechas_validas = df['fecha'].dropna()

    # Indicadores adicionales
    autores_unicos = df['autor'].nunique()
    proporcion_articulos_por_dia = df.groupby('fecha').size() / total

    # === RESULTADOS ===
    print("=== Indicadores de calidad de los datos ===")
    print(f"Total de artículos: {total}")
    print(f"Porcentaje sin autor: {nulos_autor / total * 100:.2f}%")
    print(f"Porcentaje sin título: {nulos_titulo / total * 100:.2f}%")
    print(f"Títulos duplicados: {duplicados_titulo}")
    print(f"Rango de fechas reales: {fechas_validas.min()} → {fechas_validas.max()}")
    print(f"Cantidad de autores únicos: {autores_unicos}")
    print("Proporción de artículos por día:")
    print(proporcion_articulos_por_dia)
    print("\nPrimeros artículos extraídos:")
    print(df.head())

else:
    print("Error:", data.get('message'))

=== Indicadores de calidad de los datos ===
Total de artículos: 13
Porcentaje sin autor: 0.00%
Porcentaje sin título: 0.00%
Títulos duplicados: 0
Rango de fechas reales: 2025-03-16 → 2025-03-20
Cantidad de autores únicos: 11
Proporción de artículos por día:
fecha
2025-03-16    0.076923
2025-03-17    0.230769
2025-03-18    0.076923
2025-03-19    0.153846
2025-03-20    0.461538
dtype: float64

Primeros artículos extraídos:
        fecha                                             titulo  \
0  2025-03-20  Díaz lanza un 'zasca' a Montero: "Cuando algui...   
1  2025-03-20  Yolanda Díaz hace una exigencia a Pedro Sánche...   
2  2025-03-20  Yolanda Díaz asegura que este partido del Cong...   
3  2025-03-20  Women Economic Forum 2025 va por acciones que ...   
4  2025-03-20  Imóveis: como o Brasil quer fisgar mais invest...   

               autor  
0    Autor Redacción  
1    Autor Redacción  
2    Autor Redacción  
3  citlallin.andrade  
4    Sergio Ruiz Luz  


## JUSTIFICACION DE LOS INDICADORES DE CALIDAD ###

### Total de artículos: 
Esto indicará que tan relevante fue el tema en el periodo seleccionado

### Porcentaje sin autor: 
Esto indicará que tan confiable son las noticias

### Porcentaje sin título: 
Esto indicará que tan confiable son las noticias

### Títulos duplicados: 
Esto evitará sobreestimar la cantidad de articulos

### Rango de fechas reales: 
Esto indicará el rango real de fechas consideradas

### Cantidad de autores únicos: 
Esto evitará sobreestimar la cantidad de articulos que hablas sobre el tema

### Proporción de artículos por día: 
Esto indicará cuales fueron los dias con mayor importancia a considerar para la demanda

# TIPO DE CAMBIO

In [2]:
import requests
from datetime import datetime, timedelta
import pandas as pd

# Función para generar las fechas en el rango
def generar_fechas(inicio, fin):
    fecha_inicio = datetime.strptime(inicio, "%Y-%m-%d")
    fecha_fin = datetime.strptime(fin, "%Y-%m-%d")
    delta = timedelta(days=1)
    fechas = []
    while fecha_inicio <= fecha_fin:
        fechas.append(fecha_inicio.strftime("%Y-%m-%d"))
        fecha_inicio += delta
    return fechas

# Rango de fechas deseado
fecha_inicio = "2025-04-01"
fecha_fin = "2025-04-04"
fechas_esperadas = generar_fechas(fecha_inicio, fecha_fin)

# Lista para almacenar los resultados
resultados = []

# Consultar la API para cada fecha
for fecha in fechas_esperadas:
    url = f"https://free.e-api.net.pe/tipo-cambio/{fecha}.json"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            resultados.append({
                "fecha": data.get("fecha"),
                "sunat": data.get("sunat"),
                "compra": data.get("compra"),
                "venta": data.get("venta")
            })
        else:
            print(f"{fecha}: Error {response.status_code}")
    except Exception as e:
        print(f"{fecha}: Error al consultar la API - {e}")

# Crear DataFrame con los resultados
df = pd.DataFrame(resultados)

# Convertir la columna 'fecha' a tipo datetime
df["fecha"] = pd.to_datetime(df["fecha"])

# Mostrar el DataFrame
print("\nDataFrame de tipo de cambio:")
print(df)

# -------------------------------
# Indicadores de calidad de datos
# -------------------------------
print("\n--- Indicadores de calidad de datos ---")

# 1. Verificar valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())

# 2. Fechas duplicadas
duplicadas = df["fecha"].duplicated().sum()
print(f"\nFechas duplicadas: {duplicadas}")

# 3. Rango de tipo de cambio (valores fuera de límites esperados)
fuera_de_rango = df[
    (df["compra"] < 2) | (df["compra"] > 6) |
    (df["venta"] < 2)  | (df["venta"] > 6) |
    (df["sunat"] < 2)  | (df["sunat"] > 6)
]
print(f"\nFilas con valores fuera del rango [2, 6]: {len(fuera_de_rango)}")
if not fuera_de_rango.empty:
    print(fuera_de_rango)

# 4. Días faltantes (comparar con rango original)
fechas_obtenidas = df["fecha"].dt.strftime("%Y-%m-%d").tolist()
faltantes = set(fechas_esperadas) - set(fechas_obtenidas)
print(f"\nFechas faltantes: {faltantes if faltantes else 'Ninguna'}")



DataFrame de tipo de cambio:
       fecha   sunat  compra  venta
0 2025-04-01  3.6490   3.647  3.651
1 2025-04-02  3.6685   3.660  3.677
2 2025-04-03  3.6715   3.667  3.676
3 2025-04-04  3.6755   3.671  3.680

--- Indicadores de calidad de datos ---

Valores nulos por columna:
fecha     0
sunat     0
compra    0
venta     0
dtype: int64

Fechas duplicadas: 0

Filas con valores fuera del rango [2, 6]: 0

Fechas faltantes: Ninguna


## JUSTIFICACION DE LOS INDICADORES DE CALIDAD

### Valores nulos por columna: 
Esto indicará si hay valores faltantes en alguna fecha
### Fechas duplicadas: 
Esto evitara fallas al referenciar el tipo de cambio en alguna fecha
### Filas con valores fuera del rango [2, 6]: 
Esto asegura que el tipo de cambio este en un rango razonable, si no es asi, puede tratarse de un error en la extraccion de datos
### Fechas faltantes: 
Esto asegura tener el tipo de cambio en todas las fechas esperadas en la consulta.


# TEMPERATURA

In [3]:
import requests
import pandas as pd

def obtener_temperatura(lat, lon, fecha_inicio, fecha_fin):
    url = "https://archive-api.open-meteo.com/v1/archive"
    
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": fecha_inicio,
        "end_date": fecha_fin,
        "hourly": "temperature_2m",
        "timezone": "auto"
    }

    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        return {"error": "Error en la respuesta de la API", "codigo": response.status_code}

    data = response.json()
    temperaturas = data.get("hourly", {}).get("temperature_2m", [])
    timestamps = data.get("hourly", {}).get("time", [])

    if not temperaturas or not timestamps:
        return {"error": "No se encontraron datos de temperatura"}

    df = pd.DataFrame({
        "timestamp": timestamps,
        "temperatura_2m": temperaturas
    })

    # --- Indicadores de calidad ---
    calidad = {}
    calidad["total_registros"] = len(df)
    calidad["registros_nulos"] = df["temperatura_2m"].isna().sum()
    calidad["temperatura_min"] = df["temperatura_2m"].min()
    calidad["temperatura_max"] = df["temperatura_2m"].max()

    # Valores extremos para alertar si hay datos sospechosos
    if df["temperatura_2m"].min() < -30 or df["temperatura_2m"].max() > 50:
        calidad["advertencia_valores_extremos"] = True
    else:
        calidad["advertencia_valores_extremos"] = False

    return {"datos": df, "calidad": calidad}

# Ejemplo: Lima (latitud, longitud) y rango de fechas
resultado = obtener_temperatura(-12.0464, -77.0428, "2025-03-25", "2025-03-31")

if "error" in resultado:
    print("Error:", resultado["error"])
else:
    print("Indicadores de calidad:")
    for k, v in resultado["calidad"].items():
        print(f" - {k}: {v}")
    print("\nPrimeras filas de datos:")
    print(resultado["datos"].head())


Indicadores de calidad:
 - total_registros: 168
 - registros_nulos: 0
 - temperatura_min: 19.5
 - temperatura_max: 25.8
 - advertencia_valores_extremos: False

Primeras filas de datos:
          timestamp  temperatura_2m
0  2025-03-25T00:00            20.6
1  2025-03-25T01:00            20.1
2  2025-03-25T02:00            20.1
3  2025-03-25T03:00            20.0
4  2025-03-25T04:00            19.8


## JUSTIFICACION DE LOS INDICADORES DE CALIDAD

### total_registros: 
Esto asegura tener la cantidad de registros esperados
### registros_nulos:
Esto asegura que no falte ningun registro
### advertencia_valores_extremos: 
Esto ayuda a identificar errores si los valores estan fuera de un rango razonable